In [3]:
import json
import os
from openai import OpenAI

from dotenv import load_dotenv
load_dotenv()

from openAI_utils import checkFT

In [54]:
client = OpenAI(api_key=os.getenv("OAI_KEY"))

In [5]:
def parseJSON(path="../models/mimic-iii/train.json", limit=None):
    f = open(path)
    data = json.load(f)

    limit = len(data) if limit == None else limit

    result = [(e['question'], e['query']) for e in data]
    return result[0:limit]

In [6]:
def toOpenAI(msgPairs, finetune_path="../models/mimic-iii/finetune.jsonl"):
    system_prompt = "You are a computer that provides SQL queries."
    openMessages = [{"messages": [{"role": "system", "content": system_prompt}, {"role": "user", "content": m[0]}, {"role": "assistant", "content": m[1]}]} for m in msgPairs]

    with open(finetune_path, 'w') as outfile:
        for entry in openMessages:
            json.dump(entry, outfile)
            outfile.write('\n')
    
    return openMessages

In [9]:
def toVec(msgPairs, finetune_path="../models/mimic-iii/qqdict.json"):
    openMessages = {i: msgPairs[i] for i in range(len(msgPairs))}

    with open(finetune_path, 'w') as outfile:
        json.dump(openMessages, outfile)
    
    return openMessages

toVec(parseJSON())

{0: ('tell me the method of intake of clobetasol propionate 0.05% ointment?',
  "select distinct prescriptions.route from prescriptions where prescriptions.drug = 'clobetasol propionate 0.05% ointment'"),
 1: ('what are the methods of consumption of send 500mg vial?',
  "select distinct prescriptions.route from prescriptions where prescriptions.drug = 'send 500mg vial'"),
 2: ('tell me the intake method for triamcinolone acetonide 0.1% cream?',
  "select distinct prescriptions.route from prescriptions where prescriptions.drug = 'triamcinolone acetonide 0.1% cream'"),
 3: ('what is the ingesting method of methimazole?',
  "select distinct prescriptions.route from prescriptions where prescriptions.drug = 'methimazole'"),
 4: ('what are the ways to consume caspofungin?',
  "select distinct prescriptions.route from prescriptions where prescriptions.drug = 'caspofungin'"),
 5: ('what is the intake method of the phentolamine mesylate?',
  "select distinct prescriptions.route from prescriptio

In [65]:
finetune_path="../models/mimic-iii/finetune.jsonl"
om = toOpenAI(parseJSON(), finetune_path)

checkFT(om)

No errors found


In [63]:
client.files.create(
  file=open(finetune_path, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-RwTED2DFZvCzWaPwbAEDRgb0', bytes=112244, created_at=1707614276, filename='finetune.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [64]:
client.fine_tuning.jobs.create(
  training_file="file-RwTED2DFZvCzWaPwbAEDRgb0", 
  model="gpt-3.5-turbo"
)

BadRequestError: Error code: 400 - {'error': {'message': 'Fine-tuning jobs cannot be created on an Explore plan. You can upgrade to a paid plan on your billing page: https://platform.openai.com/account/billing/overview', 'type': 'invalid_request_error', 'param': None, 'code': 'exceeded_quota'}}